# Obtain the Stable Solution
In this notebook we will recreate a version of Fig. 3 of the paper "Model Reduction of Swing Equations with Physics Informed PDE" __[arxiv](https://arxiv.org)__.

This will include the following steps:
- Load the border file and create the continuous model from the discrete data
- Calculate the stable solution
- Plot the stable solution
- Compare it to the stable solution of the discrete model



In [ ]:
# load ContGridMod package
using ContGridMod
using Plots

We start by loading the border and creating the mesh on which we run the simulation. The mesh is scaled such that the longest dimension has length 1. To convert from the original coordinates `scale_factor` can be used.

In [ ]:
# load the European border file
border, scale_factor = import_border("../data/borders/border.json");

In [ ]:
# create the mesh
dx = 0.005
mesh = get_mesh(border, dx);

To create the continuous model we need to disaggregate the values of the discrete model. For this we use artificial diffusion as described in the paper. The parameters to control the diffusion are `Niter`: number of integration steps, `dmax`: maximum distance a point can be from a line in the discrete model to still be consider on-line for the continuous model, `patch`: scaling factor for the values of $b_x$ and $b_y$ to ensure correct dimensionality, `bmin`: minimum amount for $b_x$ and $b_y$; required for numerical stability, `min_factor`: fraction of the maximum amount of inertia, damping and power that is at least distributed at every point to ensure numerical stability

In [ ]:
# distribute parameters
contmod = get_params(mesh, scale_factor, "../data/disc/pantagruel.h5", Niter=10, dmax=2*dx, patch=1900.0, bmin=52.0, tau=5.0e-7);

The stable solution can be obtained by one function call. `interval` controls at which steps the current precision is printed. The precision is the difference between two iterations, `Niter` the number of iterations, `tol` the precision at which the loop will terminate if `Niter` has not yet been reached

In [ ]:
# compute the stable solution
compute_stable_sol!(contmod, interval=Int64(1e4), Niter=Int64(2.5e5), tol=1e-7);

The stable solution can easily be plotted using the `hm_plot` command that takes care of cutting of values that are outside of the border.

In [ ]:
# plot the results
hm_plot(contmod, contmod.th, borders=[border])

To create a comparison between our continuous results and the discrete results we first load the discrete model and find the theta values of the points of the continuous model that are closest to nodes in the discrete model.

In [ ]:
# load discrete model
dm = load_discrete_model("../data/disc/pantagruel.h5", scale_factor);

In [ ]:
# Get the theta values of the stable solution closest to a node in the discrete model
th_disc = get_discrete_values(contmod.coord[contmod.isgrid,:], dm.coord, contmod.th[contmod.isgrid]);

For comparison plot the two results with equal color limits. Discrete results can be plotted using `disc_plot`.

In [ ]:
clim = extrema([contmod.th; dm.th])
plot(hm_plot(contmod, contmod.th, borders=[border],clim=clim), disc_plot(dm.coord, dm.th, borders=[border], clim=clim, markersize=5.0), layout=(2,1), aspect_ratio=:equal, size=(400,600))

In [ ]:
scatter(dm.th, th_disc, label=:none)
plot!([1.1 * minimum(dm.th); 1.1 * maximum(dm.th)], [1.1 * minimum(dm.th); 1.1 * maximum(dm.th)], label=:none, lw=3, color=:red)

Plotting in Julia feels still half baked. For more complex plots it is easier to just use Matplotlib and python. This can easily be integrated in Julia as shown in the example below.

In [ ]:
import PyPlot
using PyCall

CMAP = :inferno
contcols = fill!(Vector{Float64}(undef, contmod.Ny * contmod.Nx), NaN)
contcols[contmod.isinside] = contmod.th[contmod.isinside]
thmin = minimum(dm.th)
thmax = maximum(dm.th)
clim = extrema([dm.th;th_disc])
fig = PyPlot.plt.figure(constrained_layout=true, figsize=(8,6))
gs = fig.add_gridspec(3,2, height_ratios=[1,1,0.05], width_ratios=[1,1])
p1 = fig.add_subplot(gs[1,1], label="(a)")
p1.plot(border[:,1], border[:,2], "k", zorder=0, lw=2)
p1.scatter(dm.coord[:,2], dm.coord[:,1], c=dm.th, cmap=CMAP, zorder=1, vmin=clim[1], vmax=clim[2], s=6)
p1.set_aspect("equal")
p1.axis("off")
p1.text(0.03,0.97, "(a)", transform=p1.transAxes)
p2 = fig.add_subplot(gs[2,1])
p2.scatter(contmod.coord[:,2], contmod.coord[:,1], c=contcols, cmap=CMAP, s=3, vmin=clim[1], vmax=clim[2])
p2.plot(border[:,1], border[:,2], "k", lw=2)
p2.set_aspect("equal")
p2.axis("off")
p2.text(0.03,0.97, "(b)", transform=p2.transAxes)
cbarax = fig.add_subplot(gs[3,1])
PyPlot.plt.colorbar(p1.collections[1], cax=cbarax.axes, orientation="horizontal", label="\$\\theta\$")
p3 = fig.add_subplot(py"$(gs)[:,1]") # Careful, when using PyCall the arrays have to be 0 indexed!
p3.scatter(dm.th, th_disc, edgecolors="k")
p3.grid(color="gray", linestyle="dashed", lw=0.4)
p3.set_axisbelow(true)
p3.plot([1.1*thmin, 1.1*thmax], [1.1*thmin, 1.1*thmax], "r", lw=3)
p3.set_xlim(1.1*thmin, 1.1*thmax)
p3.set_xlabel("\$\\theta_i^{\\rm disc}\$")
p3.set_ylabel("\$\\theta_i^{\\rm cont}\$")
p3.yaxis.tick_right()
p3.yaxis.set_label_position("right")
p3.text(0.03,0.97, "(c)", transform=p3.transAxes);